# Algorithme de trading

Dans cette dernière partie du projet, nous récupérons les deux modèles entraînés ainsi que les articles de presse, les posts Reddit et le cours de l'action Apple pour conseiller à notre utilisateur d'acheter ou non une action Apple avant que le marché n'ouvre.

### Renseignez la date du jour

In [2]:
from datetime import datetime, timedelta


#On demande à l'utilisateur de saisir une date

user_date_input = input("Entrez la date d'aujourd'hui (format AAAA-MM-JJ): ")
current_date = datetime.strptime(user_date_input, "%Y-%m-%d")

## Récupération des articles de presse de Forbes des 48h précédentes

In [2]:
import requests

start_date = current_date - timedelta(days=2)
end_date = current_date
parameters = {
    'q': 'Apple',
    'domains': 'forbes.com',
    'from': start_date.strftime("%Y-%m-%d"),
    'to': end_date.strftime("%Y-%m-%d"),
    'sortBy': 'publishedAt',
    'apiKey': '1b20fb6f9b9d40f0b4e4ad6fe5d90755'  # Assurez-vous d'utiliser votre propre clé API
}

url = 'https://newsapi.org/v2/everything'

# Appel de l'API
response = requests.get(url, params=parameters)
articles = response.json().get('articles', [])  # Utilisez .get pour éviter les erreurs si 'articles' n'existe pas

# Itérer sur chaque article récupéré
for article in articles:
    title = article.get('title', 'Titre indisponible')  # Utilisez .get pour éviter les erreurs si 'title' n'existe pas
    source_name = article.get('source', {}).get('name', 'Source indisponible')  # Récupérer le nom de la source
    url = article.get('url', 'URL indisponible')  # Récupérer l'URL de l'article
    print(f"Titre: {title}\nSource: {source_name}\nURL: {url}\n")

/Users/dabadieraphael/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Titre: Evaluating Industrials In The 2I Era: A Closer Look
Source: Forbes
URL: https://www.forbes.com/sites/forbesbusinesscouncil/2023/12/26/evaluating-industrials-in-the-2i-era-a-closer-look/

Titre: 3 Improv Games To Get Your Team Out Of Their Comfort Zone
Source: Forbes
URL: https://www.forbes.com/sites/forbesagencycouncil/2023/12/26/3-improv-games-to-get-your-team-out-of-their-comfort-zone/

Titre: What Enterprises Should Know About ESIM Technology
Source: Forbes
URL: https://www.forbes.com/sites/forbestechcouncil/2023/12/26/what-enterprises-should-know-about-esim-technology/

Titre: Samsung Insiders Confirm Three Stunning Galaxy S24 Ultra Decisions
Source: Forbes
URL: https://www.forbes.com/sites/ewanspence/2023/12/25/samsung-galaxy-s24-ultra-design-performance-launch-exynos-snapdragon-new-galaxy-s24-release/

Titre: Apple iOS 17.3 Release Date: Awesome iPhone Upgrade Coming Soon
Source: Forbes
URL: https://www.forbes.com/sites/davidphelan/2023/12/25/ios-173-apple-is-about-to-laun

### Nettoyage des articles

In [3]:
import csv
import pandas as pd

with open('articles_algo.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Titre', 'URL', 'Date de publication', 'Description','Source'])
    for article in articles:
        if "Apple" not in article['title']:
          continue
        writer.writerow([article['title'], article['url'], article['publishedAt'], article['description'], article['source']['name']])

df=pd.read_csv('articles_algo.csv')

urls = [url for url in df['URL']]
html_contents = []
compteur=0
df['Contenu']=[None] * len(df)

for url in urls:
    response = requests.get(url)
    if response.status_code == 200:
        html_contents.append(response.text)
        df['Contenu'].loc[compteur]=response.text
    else:
        print(f"Échec de récupération pour {url}")
    compteur+=1

df.head()

,Titre,URL,Date de publication,Description,Source,Contenu
0,Apple iOS 17.3 Release Date: Awesome iPhone Up...,https://www.forbes.com/sites/davidphelan/2023/...,2023-12-25T21:26:53Z,The next big iOS update is on its way and incl...,Forbes,"<!DOCTYPE html><html lang=""en""><head><link rel..."


In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
from bs4 import BeautifulSoup

def cleaning_text(text):
    #Passage du texte en miniscules
    text=text.lower()
    #Suppression des chiffres
    text=re.sub(r'\d+', '', text)
    #Suppression de la ponctuation et des symboles spéciaux
    text=re.sub(r'[^\w\s]', '', text)
    return text


stops = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def clean_stopwords(text):
    words = word_tokenize(text)
    cleaned_text=[word for word in words if word not in stops]
    return ' '.join(cleaned_text)

#Fonction adaptée à la structure des articles Forbes
def clean_html_1(text_html): 
    soup=BeautifulSoup(text_html, 'html.parser')
    title=soup.find_all('h1', class_=True)
    content=soup.find_all('p')
    united_content = ' '.join(el.get_text(strip=True) for el in title + content )
    return united_content


In [6]:
if len(df)>0:
    df['Cleaned_html_content'] = df.apply(lambda row: clean_html_1(row['Contenu']), axis=1)
    df['Content_cleaned']=df['Cleaned_html_content'].apply(cleaning_text)
    df['Content_cleaned_from_stopwords']=df['Content_cleaned'].apply(clean_stopwords)

,Titre,URL,Date de publication,Description,Source,Contenu,Cleaned_html_content,Content_cleaned,Content_cleaned_from_stopwords
0,Apple iOS 17.3 Release Date: Awesome iPhone Up...,https://www.forbes.com/sites/davidphelan/2023/...,2023-12-25T21:26:53Z,The next big iOS update is on its way and incl...,Forbes,"<!DOCTYPE html><html lang=""en""><head><link rel...",Apple iOS 17.3 Release Date: Awesome iPhone Up...,apple ios release date awesome iphone upgrade...,apple ios release date awesome iphone upgrade ...


### Analyse de sentiments avec un modèle pré-entraîné

In [12]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis')

sentiments=[]
#Comme nous l'avions vu précédemment, il faut tronquer les textes pour les réduire à 512 tokens au risque d'obtenir un message d'erreur
for i in range(len(df)):
    text = df.iloc[i,8]
    text = text[:512]
    sentiment = classifier(text)[0]['label']  
    sentiments.append(sentiment)
df['predicted_sentiment'] = sentiments
df.head()



No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


,Titre,URL,Date de publication,Description,Source,Contenu,Cleaned_html_content,Content_cleaned,Content_cleaned_from_stopwords,predicted_sentiment
0,Apple iOS 17.3 Release Date: Awesome iPhone Up...,https://www.forbes.com/sites/davidphelan/2023/...,2023-12-25T21:26:53Z,The next big iOS update is on its way and incl...,Forbes,"<!DOCTYPE html><html lang=""en""><head><link rel...",Apple iOS 17.3 Release Date: Awesome iPhone Up...,apple ios release date awesome iphone upgrade...,apple ios release date awesome iphone upgrade ...,NEGATIVE


On récupère le pourcentage d'articles à connotation positive parmi les articles qu'on a récupérés.

In [17]:
rate_pos = (df['predicted_sentiment'] == 'POSITIVE').mean()

print(f"Taux de prédictions 'POSITIVE': {rate_pos:}")


Taux de prédictions 'POSITIVE': 0.0


## Récupération des posts Reddit

## Récupération des informations de Yahoo Finance 

On l'a déja vu dans le fichier dédié.

In [8]:
import matplotlib.pyplot as plt 
import numpy as np
import yfinance as yf 
import pandas as pd 
from datetime import datetime

APL = "AAPL"
data = yf.Ticker(APL) # Extraction avec la fonction yf.Ticker de yfinance
prix_rec = data.history(period = '1d', start = '2020-1-1', end = current_date)
prix_rec.to_csv('AAPLt.csv')
df=pd.read_csv("AAPLt.csv") 
df['Date'] = df['Date'].astype(str)
df['Date'] = df['Date'].str.slice(0, 10)
df['Date'] = pd.to_datetime(df['Date'])
df = df[['Date', 'Close']]
df

,Date,Close
0,2020-01-02,73.152641
1,2020-01-03,72.441460
2,2020-01-06,73.018684
3,2020-01-07,72.675278
4,2020-01-08,73.844353
...,...,...
1000,2023-12-21,194.679993
1001,2023-12-22,193.600006
1002,2023-12-26,193.050003
1003,2023-12-27,193.149994


### Prédiction de la valeur du lendemain 

On a vu dans la partie dédiée que le modèle le plus efficace était le naive forecast.
Cependant, par définition du modèle, la valeur du lendemain que le modèle va prédire est la valeur réelle d'aujourd'hui. Donc il ne sera pas possible de tirer de conclusion sur oui ou non il faut acheter l'action, car sa valeur prédite sera la même qu'aujourd'hui.

Pour résoudre ce problème, nous allons comparer la valeur prédite, donc la valeur à J+1, avec la valeur à J-1.
Si la valeur est supérieure, alors nous considérerons qu'il faut acheter. 
Sinon, il vaut mieux ne pas acheter.
Une autre solution proposée est d'utiliser quand même le modèle LTSM, bien qu'il soit moins précis que le modèle de naive forecast, mais qui ne nécessite pas de recourir à ce compromis.

La valeur prédite du lendemain est donc la valeur de l'action au jour d'input, c'est à dire la dernière valeur de fermeture du marché :

In [12]:
valeur_lendemain = df['Close'].iloc[-1]
valeur_lendemain

193.85499572753903

Comparons la a la valeur du jour J-1, qui est donc :

In [13]:
valeur_prec = df['Close'].iloc[-2]
valeur_prec

193.1499938964844

In [15]:
def acheter_ou_non():
    if valeur_lendemain > valeur_prec :
        return 'POSITIVE'
    else : 
        return 'NEGATIVE'
    
acheter_ou_non()

'POSITIVE'